In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from google.colab import drive
drive.mount('/content/drive')

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.metrics import Precision, Recall
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0(weights='imagenet')

Mounted at /content/drive
21834768/21834768 [==============================] - 0s 0us/step


In [ ]:
model = EfficientNetB0(include_top=False, weights='imagenet')
model = EfficientNetB0(weights='imagenet', drop_connect_rate=0.5)


16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
IMG_SIZE = 224
dataset_path = '/content/drive/MyDrive/KIMIA_Path_960_Complete'

In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory('/content/drive/MyDrive/KIMIA_Path_960_Complete', target_size=(224, 224))

Found 960 images belonging to 20 classes.


In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf  # For tf.data
import matplotlib.pyplot as plt
import keras
from keras import layers

In [ ]:
img_augmentation_layers = [
    layers.RandomRotation(factor=0.15),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomFlip(),
    layers.RandomContrast(factor=0.1),
]


def img_augmentation(images):
    for layer in img_augmentation_layers:
        images = layer(images)
    return images

In [ ]:
model = EfficientNetB0(
    include_top=True,
    weights=None,
    classes=20,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

epochs = 40  # @param {type: "slider", min:10, max:100}
hist = model.fit(traindata, epochs=20)

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_6 (Rescaling)     (None, 224, 224, 3)          0         ['input_4[0][0]']             
                                                                                                  
 normalization_3 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_6[0][0]']         
 tion)                                                                                            
                                                                                                  
 stem_conv_pad (ZeroPadding  (None, 225, 225, 3)          0         ['normalization_3

In [ ]:
def build_model(num_classes):
    inputs = layers.Input(shape=(224, 224, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(256, activation = "relu", name="dense")(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=20)
model.summary()
resultModel = model.fit(traindata, epochs=20, steps_per_epoch=len(traindata))


Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_17 (Rescaling)    (None, 224, 224, 3)          0         ['input_10[0][0]']            
                                                                                                  
 normalization_9 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_17[0][0]']        
 tion)                                                                                            
                                                                                                  
 rescaling_18 (Rescaling)    (None, 224, 224, 3)          0         ['normalization_9[0

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

epochs = 20
hist = model.fit(traindata, epochs=20)

Epoch 1/20
30/30 [==============================] - 14s 153ms/step - loss: 0.6172 - accuracy: 0.9531
Epoch 2/20
30/30 [==============================] - 6s 182ms/step - loss: 0.5642 - accuracy: 0.9646
Epoch 3/20
30/30 [==============================] - 5s 155ms/step - loss: 0.3223 - accuracy: 0.9719
Epoch 4/20
30/30 [==============================] - 5s 161ms/step - loss: 0.5578 - accuracy: 0.9573
Epoch 5/20
30/30 [==============================] - 5s 171ms/step - loss: 0.4617 - accuracy: 0.9656
Epoch 6/20
30/30 [==============================] - 5s 154ms/step - loss: 0.2186 - accuracy: 0.9781
Epoch 7/20
30/30 [==============================] - 6s 217ms/step - loss: 0.2568 - accuracy: 0.9729
Epoch 8/20
30/30 [==============================] - 5s 160ms/step - loss: 0.2816 - accuracy: 0.9750
Epoch 9/20
30/30 [==============================] - 5s 162ms/step - loss: 0.1808 - accuracy: 0.9844
Epoch 10/20
30/30 [==============================] - 5s 174ms/step - loss: 0.2216 - accuracy: 0.978

In [ ]:
import numpy as np
import os
import cv2
import pandas as pd

dense_layer = 'dense'
feature_model = Model(inputs = model.input, outputs = model.get_layer(dense_layer).output)

image_size = 224
features_list = []
filenames_list = []

labels = ['Class_A','Class_B','Class_C','Class_D','Class_E','Class_F','Class_G','Class_H','Class_I','Class_J','Class_K','Class_L','Class_M','Class_N','Class_O','Class_P','Class_Q','Class_R','Class_S','Class_T']
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/KIMIA_Path_960_Complete',i)
    for j in sorted(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))

        features = feature_model.predict(np.expand_dims(img, axis=0))
        filenames_list.append(i)
        features_list.append(features.flatten())

features_df = pd.DataFrame(features_list)
features_df['Filename'] = filenames_list

features_df.to_excel('efficientnet_features1.xlsx', index = False)